In [1]:
#  import all from generator.imports
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../generator/')

from generator.imports import *

from db import *

In [2]:
# !pip install psycopg2

# Test reading

In [3]:
# create postgress connection

from sqlalchemy import create_engine

engine = create_engine('postgresql://{}:{}@{}/{}'.format(PG_USER, PG_PASSWORD, PG_HOST, DB_NAME))



In [4]:
import sqlalchemy
from sqlalchemy import text
sqlalchemy.__version__

'2.0.0'

In [5]:
# create a table in pg with name test using sqlalchemy
with engine.connect() as conn:
    
    sql = """DROP TABLE IF EXISTS mfg.test;"""
    conn.execute(text(sql))
    
    
    # create a new schema with name mfg in pg
    sql = """CREATE SCHEMA IF NOT EXISTS mfg;"""
    conn.execute(text(sql))
    
    
    sql = """CREATE TABLE mfg.test (ID SERIAL PRIMARY KEY, value VARCHAR(255));"""
    conn.execute(text(sql))
    
    conn.commit()

In [6]:
# read data from postgres

with engine.connect() as conn:

    df = pd.read_sql_table('test', con = conn, schema='mfg')

In [7]:
# define a function to refresh data to postgres

def write_to_postgres(df, table_name, schema_name, engine):
    
    with engine.connect() as conn:
        
        df.to_sql(table_name, con=conn, schema=schema_name, if_exists='replace', index=False)
        
        conn.commit()

In [8]:
# read all excel and csv files from BASE_DIR and write to postgres
def write_to_db(dir, file_name = None):
    
    if file_name is None:
        
        for file in os.listdir(dir):
            if file.endswith('.xlsx') or file.endswith('.csv'):
                print('read {} to postgres'.format(file))
            
                # read excel file
                if file.endswith('.xlsx'):
                    df = pd.read_excel(os.path.join(dir, file))
                
                if file.endswith('.csv'):
                    df = pd.read_csv(os.path.join(dir, file))
                
            
                write_to_postgres(df, file.split('.')[0], 'mfg', engine)
                
                print('write {} to postgres'.format(file.split('.')[0]))

    elif file_name is not None:
        
        # read excel file
        if file_name.endswith('.xlsx'):
            df = pd.read_excel(os.path.join(dir, file_name))
        
        if file_name.endswith('.csv'):
            df = pd.read_csv(os.path.join(dir, file_name))
            
        write_to_postgres(df, file_name.split('.')[0], 'mfg', engine)
        
        print('write {} to postgres'.format(file_name.split('.')[0]))


def write_to_db_df(df, table_name):
    
    write_to_postgres(df, table_name, 'mfg', engine)
        
    print('write {} to postgres'.format(table_name))

# Write all to db

In [11]:
write_to_db(GE_DIR, 'order_detail_map.csv')

write order_detail_map to postgres


In [13]:
# write_to_db(GE_DIR, 'parts_order_inventory.csv')

write parts_order_inventory to postgres


In [25]:
# write_to_db(BASE_DIR)
# write_to_db(GE_DIR)

# write certain file to postgres

In [26]:
write_to_db(GE_DIR, 'order_monthly.csv')

write order_monthly to postgres


In [31]:
write_to_db(BASE_DIR, 'sites.csv')

write sites to postgres


In [47]:
# write_to_db(GE_DIR, 'order_detail.csv')
write_to_db(GE_DIR, 'cycle_time.csv')

write cycle_time to postgres


In [ ]:
# write All base adata to postgres

# Map data

In [4]:
df_map = pd.read_csv('/Users/zhitaogao/Library/CloudStorage/OneDrive-Personal/bbd-projects/mfg_operation_model/src/map_china.csv')

# Map

In [13]:
write_to_db_df(df_map, 'map_china')

write map_china to postgres


# Map D3

In [20]:
path = "/Users/zhitaogao/Library/CloudStorage/OneDrive-Personal/bbd-projects/mfg_operation_model/map_generation_src/map_china_d3.csv"

df_d3 = pd.read_csv(path)

In [21]:
write_to_db_df(df_d3, 'map_china_d3')

write map_china_d3 to postgres


# 预测性维护 Equipment Status

In [15]:
name = "equipment_vibration.csv"
df = pd.read_csv(os.path.join(GE_DIR, name))
df

,noise_data,time
0,8.554032,2023-04-24 21:26:47.247329
1,11.735937,2023-04-24 21:26:47.257329
2,10.042567,2023-04-24 21:26:47.267329
3,9.008329,2023-04-24 21:26:47.277329
4,10.893289,2023-04-24 21:26:47.287329
...,...,...
359996,13.171651,2023-04-24 22:26:47.207329
359997,15.284330,2023-04-24 22:26:47.217329
359998,14.293328,2023-04-24 22:26:47.227329
359999,14.149797,2023-04-24 22:26:47.237329


In [22]:
write_to_db_df(df, name.split('.')[0])

write equipment_vibration to postgres


In [25]:
name = "equipment_vibration_rolling_std.csv"
df = pd.read_csv(os.path.join(GE_DIR, name))
df

,level_0,index,rolling_std,noise_data,time
0,0,0,NaN,8.554032,2023-04-24 21:26:47.247329
1,1,1000,0.968799,9.215418,2023-04-24 21:26:57.247329
2,2,2000,0.989600,9.113201,2023-04-24 21:27:07.247329
3,3,3000,0.986585,8.932201,2023-04-24 21:27:17.247329
4,4,4000,1.027769,9.666576,2023-04-24 21:27:27.247329
...,...,...,...,...,...
356,356,356000,1.410122,15.487204,2023-04-24 22:26:07.247329
357,357,357000,1.470979,15.099061,2023-04-24 22:26:17.247329
358,358,358000,1.410370,14.643481,2023-04-24 22:26:27.247329
359,359,359000,1.412678,15.949526,2023-04-24 22:26:37.247329


In [27]:
write_to_db_df(df, name.split('.')[0])

write equipment_vibration_rolling_std to postgres
